In [1]:
import pandas as pd
import numpy as np
import cv2
import dlib
import os

In [2]:
face_cascade = cv2.CascadeClassifier('../face_recognition/haarcascade_frontalface_default.xml')
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

In [5]:
IMG_PATH="../facial_expressions/"

In [184]:
emotions=["anger", "disgust", "fear", "happy", "sad", "surprise", "neutral"]

In [111]:
df=pd.read_csv(IMG_PATH+"icml_face_data.csv")

In [113]:
df=df.replace(to_replace="PrivateTest",value="Testing")
df=df.replace(to_replace="PublicTest",value="Training")

In [114]:
df

,emotion,Usage,pixels
0,0,Training,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...
1,0,Training,151 150 147 155 148 133 111 140 170 174 182 15...
2,2,Training,231 212 156 164 174 138 161 173 182 200 106 38...
3,4,Training,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...
4,6,Training,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...
...,...,...,...
35882,6,Testing,50 36 17 22 23 29 33 39 34 37 37 37 39 43 48 5...
35883,3,Testing,178 174 172 173 181 188 191 194 196 199 200 20...
35884,0,Testing,17 17 16 23 28 22 19 17 25 26 20 24 31 19 27 9...
35885,3,Testing,30 28 28 29 31 30 42 68 79 81 77 67 67 71 63 6...


In [115]:
data={"Training":[],"Testing":[]}

In [116]:
for i in range(len(df)):
    print("\r",i+1,end=" ")
    row=df.iloc[i]
    img=np.array(list(map(np.uint8,row[2].split()))).reshape(48,48)
    data[row[1]].append([row[0],ret_keypoints(img)])

 35887  4965      32797  

In [138]:
import pickle

In [139]:
with open("data.pkl","wb") as f:
    pickle.dump(data, f)

In [ ]:
# with open('data.pkl', 'rb') as f:
#     data = pickle.load(f)

In [140]:
face_dim=64

In [141]:
import matplotlib.pyplot as plt

In [142]:
def ret_keypoints(image):
    gray=image
#     gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = cv2.resize(gray, (face_dim, face_dim))
#     faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    param=[0,0,face_dim,face_dim]
    shape=predictor(gray,dlib.rectangle(*param))
    xlist=[]
    ylist=[]
    for i in range(68):
        xlist.append(np.float32(shape.part(i).x))
        ylist.append(np.float32(shape.part(i).y))
    xmean = np.mean(xlist)
    ymean = np.mean(ylist)
#     plt.imshow(gray,cmap='gray')
#     plt.scatter(xlist,ylist, marker='.')
#     plt.show()
    xcentral = [(x-xmean) for x in xlist]
    ycentral = [(y-ymean) for y in ylist]
    res=[]
    for (x,y) in zip(xcentral,ycentral):
        res.append(x)
        res.append(y)
    return (np.asarray(res)/face_dim+1)/2

In [143]:
# s=np.arange(len(XXD))
# for i in range(5):
#     np.random.shuffle(s)
#     XXD=XXD[s]
#     YYD=YYD[s]

In [144]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization

In [192]:
model=Sequential()
model.add(Dense(1024,activation='relu', input_shape=(68*2,)))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(512,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(256,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.4))
model.add(Dense(128,activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(len(emotions),activation='softmax'))

In [193]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 1024)              140288    
_________________________________________________________________
batch_normalization_4 (Batch (None, 1024)              4096      
_________________________________________________________________
dropout_4 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_6 (Dense)              (None, 512)               524800    
_________________________________________________________________
batch_normalization_5 (Batch (None, 512)               2048      
_________________________________________________________________
dropout_5 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 256)              

In [195]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [188]:
XDB=np.vstack(np.asarray(data["Training"])[:,1])
YDY=np.asarray(data["Training"])[:,0]
YDB=np.zeros((len(YDY),len(emotions)))
for i in range(len(YDB)):
    YDB[i][YDY[i]]=1
XTB=np.vstack(np.asarray(data["Testing"])[:,1])
YTY=np.asarray(data["Testing"])[:,0]
YTB=np.zeros((len(YTY),len(emotions)))
for i in range(len(YTB)):
    YTB[i][YTY[i]]=1

In [ ]:
model.fit(XDB,YDB,batch_size=64,epochs=100,validation_data=(XTB,YTB))

Train on 32298 samples, validate on 3589 samples
Epoch 1/100
32298/32298 [==============================] - 7s 231us/sample - loss: 1.2524 - accuracy: 0.5162 - val_loss: 1.4062 - val_accuracy: 0.4678
Epoch 2/100
32298/32298 [==============================] - 8s 257us/sample - loss: 1.2488 - accuracy: 0.5179 - val_loss: 1.3018 - val_accuracy: 0.5026
Epoch 3/100
32298/32298 [==============================] - 7s 226us/sample - loss: 1.2486 - accuracy: 0.5162 - val_loss: 1.3351 - val_accuracy: 0.4809
Epoch 4/100
32298/32298 [==============================] - 7s 220us/sample - loss: 1.2479 - accuracy: 0.5199 - val_loss: 1.2618 - val_accuracy: 0.5088
Epoch 5/100
32298/32298 [==============================] - 8s 247us/sample - loss: 1.2462 - accuracy: 0.5201 - val_loss: 1.3826 - val_accuracy: 0.4673
Epoch 6/100
32298/32298 [==============================] - 8s 236us/sample - loss: 1.2484 - accuracy: 0.5172 - val_loss: 1.2854 - val_accuracy: 0.4976
Epoch 7/100
32298/32298 [====================

32298/32298 [==============================] - 7s 216us/sample - loss: 1.2200 - accuracy: 0.5287 - val_loss: 1.3201 - val_accuracy: 0.4887
Epoch 55/100
32298/32298 [==============================] - 9s 273us/sample - loss: 1.2255 - accuracy: 0.5256 - val_loss: 1.3428 - val_accuracy: 0.4812
Epoch 56/100
10304/32298 [========>.....................] - ETA: 4s - loss: 1.2224 - accuracy: 0.5285

In [201]:
(model.predict(XTB).argmax(axis=1)==YTB.argmax(axis=1)).sum()/len(XTB)

0.49874616884926165

In [202]:
model.predict(XTB).argmax(axis=1)

array([6, 0, 4, ..., 4, 4, 6])

In [203]:
YTB.argmax(axis=1)

array([0, 5, 6, ..., 0, 3, 2])

In [204]:
cam = cv2.VideoCapture(0)

while True:
    ret, img = cam.read()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    landmarks=[]
    for (x,y,w,h) in faces:
        svx,svy=x,y
        gray=gray[y:y+h,x:x+w]
        try:
            gray=cv2.resize(gray, (face_dim, face_dim))
        except:
            print(gray,x,y,w,h)
            continue
        shape=predictor(gray,dlib.rectangle(0,0,face_dim,face_dim))
        xlist=[]
        ylist=[]
        for i in range(68):
            xp=shape.part(i).x
            yp=shape.part(i).y
            cv2.circle(gray, (xp, yp), 2, (255, 255, 255), -1)
            xlist.append(float(xp))
            ylist.append(float(yp))
        cv2.imshow('gray',gray)
        xmean = np.mean(xlist)
        ymean = np.mean(ylist)
        xcentral = [(x-xmean) for x in xlist]
        ycentral = [(y-ymean) for y in ylist]
        res=[]
        for (x,y) in zip(xcentral,ycentral):
            res.append(x)
            res.append(y)
        landmarks.append((np.asarray(res)/face_dim+1)/2)
    if len(landmarks)>0:
        y_out=model.predict(np.asarray(landmarks))
        res=np.argmax(y_out,axis=1)
        for r in res:
            cv2.putText(img,emotions[r],(svx,svy),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,255,0),2,cv2.LINE_AA)
    cv2.imshow('webcam', img)
    if cv2.waitKey(1) & 0xff == 27:
        break

cam.release()
cv2.destroyAllWindows()

[] 23 237 188 188
[] 472 220 106 106
[] 468 240 116 116
[] 7 157 255 255
[] 20 162 226 226
[] 18 157 228 228
[] 22 160 223 223
[] 315 110 179 179
[] 21 162 226 226
[] 19 163 230 230
[] 19 160 228 228
[] 18 160 231 231
[] 18 155 231 231
[] 22 164 221 221
[] 18 158 226 226
[] 23 165 219 219
[] 15 154 233 233
[] 20 159 223 223
[] 21 159 221 221
[] 16 156 230 230
[] 11 156 238 238
[] 17 159 226 226
[] 18 160 226 226
[] 18 158 228 228
[] 23 164 219 219
[] 21 162 220 220
[] 21 163 224 224
[] 21 165 220 220
[] 20 162 223 223
[] 15 155 234 234
[] 19 157 226 226
[] 20 162 226 226
[] 22 161 231 231
[] 28 167 223 223
[] 27 169 223 223
[] 33 175 223 223
[] 33 183 226 226
[] 38 181 229 229
[] 43 190 223 223
[] 47 192 220 220
[] 50 192 221 221
[] 53 192 223 223
[] 48 190 227 227
[] 48 191 226 226
[] 48 189 228 228
[] 48 193 220 220
[] 50 189 216 216
[] 45 184 224 224
[] 51 188 213 213
[] 52 185 216 216
[] 51 183 216 216
[] 43 178 221 221
[] 47 175 219 219
[] 55 181 208 208
[] 48 175 216 216
[] 51 17

In [21]:
cam.release()
cv2.destroyAllWindows()